## Read in Libraries and Data

In [510]:
import pandas as pd
import numpy as np
import geopandas as gp
import requests as req
import re

In [511]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('precision', 2)

In [512]:
permits = pd.read_csv("DI Magnum NewDrill Permits.CSV")
prod = pd.read_csv("DI Magnum Production.CSV")
counties = gp.read_file("zip://UScounties.zip")

## Clean Up Production DataFrame

In [513]:
prod.head()
prod["Producing Status"].value_counts()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.
0,42469344440000,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,INACTIVE,V,9500.0,2019-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-25,NaN,3800.0,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9270.0,9312.0,0.66,46.3,2020-02-23,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",DAVID HINES,130836314,NaN,NaN,NaN,0
1,42245328640000,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,INACTIVE,V,8560.0,2018-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5961.0,6044.0,NaN,30.5,2019-06-29,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27819,83.0,NaN,BIG HILL,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,373,USA,NaN,NaN,29.76,551,-94.26,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",EL&RR RR CO,130833332,NaN,NaN,NaN,0
2,42039333450000,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-11,1,NaN,NaN,NaN,27788,NaN,NaN,WILDCAT,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,65,USA,NaN,NaN,28.90,60,-95.60,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","DILLARD, N",130819038,NaN,NaN,NaN,0
3,42321324010000,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,2017-03-08,2020-01-01,2020-04-01,4.0,19409.0,160.0,687.0,6.0,144.0,6151.0,138.0,3922.0,NaN,197.0,3206.0,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,7.0,33.0,159.0,198448.0,29.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,23820.0,28251.82,28250.0,35.4,42.0,23.4,6151.0,4.0,197.0,1.0,1169.0,4.0,7.0,4.0,8781.0,8950.0,NaN,40.0,2020-01-09,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27736,169.0,NaN,"LUCKY, EAST",TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.96,45,-95.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","HALL, E",130785484,687.0,19409.0,138.0,0
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP","L & L TRANSPORTATION, LLC",NaN,NaN,NaN,"OAKLEY, NORTHWEST",LA,NaN,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCI

INACTIVE                                  560
P & A                                     149
WORKED OVER TO ANOTHER ZONE                52
TA                                         17
RECLASSIFIED TO ANOTHER TYPE OF ENTITY     10
ACTIVE INJ                                  7
ACTIVE                                      7
CANCELLED                                   5
PERMIT                                      3
APPLICATION FOR PERMIT TO DRILL             1
SHUT IN                                     1
Name: Producing Status, dtype: int64

### Remove P&A Wells and API Trailing 0's

In [514]:
#filtering out wells that have already been P&A by magnum
prod = prod.loc[prod["Producing Status"]!='P & A']

prod["Producing Status"].value_counts()

INACTIVE                                  560
WORKED OVER TO ANOTHER ZONE                52
TA                                         17
RECLASSIFIED TO ANOTHER TYPE OF ENTITY     10
ACTIVE INJ                                  7
ACTIVE                                      7
CANCELLED                                   5
PERMIT                                      3
APPLICATION FOR PERMIT TO DRILL             1
SHUT IN                                     1
Name: Producing Status, dtype: int64

In [515]:
#testing to see if api number can be cleaned by dividing number by 10,000 to remove trailing 0's
test = prod["API/UWI"].iloc[0]

test

int(test/(10**4))


42469344440000

4246934444

In [516]:
prod[prod["API/UWI"]==0]

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP","L & L TRANSPORTATION, LLC",NaN,NaN,NaN,"OAKLEY, NORTHWEST",LA,NaN,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, L.P.",NaN,130769684,119.0,NaN,0.0,801958
84,0,"MAGNUM PRODUCING, LP",NaN,3A,WHARTON (TX),MIOCENE 4300,SWD,CANCELLED,V,4800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3700.0,4800.0,NaN,NaN,NaN,1,NaN,NaN,NaN,000105422,1100.0,NaN,LOUISE,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",NaN,128273620,NaN,NaN,NaN,0
271,0,"MAGNUM PRODUCING, LP",PAINTERS GAS UNIT,NaN,UNKNOWN (TX),3-G-1,GAS,INACTIVE,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,207020,NaN,0.0,MERCEDES,TX,04,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, LP",NaN,105561337,NaN,NaN,NaN,0
409,0,MAGNUM PRODUCING & OPERG CO,HOUSSIERE ET AL,NaN,ACADIA (LA),NaN,OIL,INACTIVE,U,NaN,NaN,1998-10-01,1998-10-01,1.0,NaN,NaN,1.0,NaN,1.0,NaN,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,EQUILON ENTERPRISES LLC,NaN,NaN,NaN,JENNINGS,LA,LAFAYETTE,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P-310,MAGNUM PRODUCING & OPERG CO,NaN,1672294,1.0,NaN,0.0,46342
776,0,MAGNUM PRODUCING & OPERATING CO.,"JACOBS, C.Y.",NaN,ANDERSON (TX),FRIO 2970,GAS,INACTIVE,V,2964.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,158866,NaN,0.0,"HELEN GOHLKE, N. E.",TX,06,EAST TEXAS BASIN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P-320,MAGNUM PRODUCING & OPERATING CO.,NaN,187470,NaN,NaN,NaN,0


In [517]:
#dividing each api by 10,000 to clean number and remove trailing 0's
prod["API/UWI"] = prod["API/UWI"].apply(lambda x: int(x/(10**4)))

In [518]:
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.
0,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,INACTIVE,V,9500.0,2019-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-25,NaN,3800.0,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9270.0,9312.0,0.66,46.3,2020-02-23,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",DAVID HINES,130836314,NaN,NaN,NaN,0
1,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,INACTIVE,V,8560.0,2018-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5961.0,6044.0,NaN,30.5,2019-06-29,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27819,83.0,NaN,BIG HILL,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,373,USA,NaN,NaN,29.76,551,-94.26,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",EL&RR RR CO,130833332,NaN,NaN,NaN,0
2,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-11,1,NaN,NaN,NaN,27788,NaN,NaN,WILDCAT,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,65,USA,NaN,NaN,28.90,60,-95.60,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","DILLARD, N",130819038,NaN,NaN,NaN,0
3,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,2017-03-08,2020-01-01,2020-04-01,4.0,19409.0,160.0,687.0,6.0,144.0,6151.0,138.0,3922.0,NaN,197.0,3206.0,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,7.0,33.0,159.0,198448.0,29.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,23820.0,28251.82,28250.0,35.4,42.0,23.4,6151.0,4.0,197.0,1.0,1169.0,4.0,7.0,4.0,8781.0,8950.0,NaN,40.0,2020-01-09,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27736,169.0,NaN,"LUCKY, EAST",TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.96,45,-95.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","HALL, E",130785484,687.0,19409.0,138.0,0
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP","L & L TRANSPORTATION, LLC",NaN,NaN,NaN,"OAKLEY, NORTHWEST",LA,NaN,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, L.P.",NaN,13

### Create Cleaned County Column
* cleaned county column only has name and no state appended at the end

In [519]:
#clean county names
prod["County/Parish"].iloc[197][:-5]

prod["Cleaned County"] = prod["County/Parish"].apply(lambda x: x[:-5])

'AUSTIN'

In [520]:
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.,Cleaned County
0,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,INACTIVE,V,9500.0,2019-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-25,NaN,3800.0,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9270.0,9312.0,0.66,46.3,2020-02-23,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",DAVID HINES,130836314,NaN,NaN,NaN,0,VICTORIA
1,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,INACTIVE,V,8560.0,2018-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5961.0,6044.0,NaN,30.5,2019-06-29,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27819,83.0,NaN,BIG HILL,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,373,USA,NaN,NaN,29.76,551,-94.26,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",EL&RR RR CO,130833332,NaN,NaN,NaN,0,JEFFERSON
2,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-11,1,NaN,NaN,NaN,27788,NaN,NaN,WILDCAT,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,65,USA,NaN,NaN,28.90,60,-95.60,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","DILLARD, N",130819038,NaN,NaN,NaN,0,BRAZORIA
3,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,2017-03-08,2020-01-01,2020-04-01,4.0,19409.0,160.0,687.0,6.0,144.0,6151.0,138.0,3922.0,NaN,197.0,3206.0,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,7.0,33.0,159.0,198448.0,29.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,23820.0,28251.82,28250.0,35.4,42.0,23.4,6151.0,4.0,197.0,1.0,1169.0,4.0,7.0,4.0,8781.0,8950.0,NaN,40.0,2020-01-09,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27736,169.0,NaN,"LUCKY, EAST",TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.96,45,-95.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","HALL, E",130785484,687.0,19409.0,138.0,0,MATAGORDA
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP","L & L TRANSPORTATION, LLC",NaN,NaN,NaN,"OAKLEY, NORTHWEST",LA,NaN,NaN,NaN,USA,NaN,NaN,Na

### Adding Latest BOE/month column

In [521]:
prod["Latest BOE"] = (prod["Latest Gas"]/6) + prod["Latest Oil"]

## Permits Table

In [522]:
permits.head()

,API 10/UWI,District,Filed Date,Approved Date,Expired Date,State/Province,County/Parish,Operator Alias (Legacy),Lease Name,Well Number,Formation,Permit Depth,True Vertical Depth,Permit Type,Well Type,Drill Type,Well Status,Permit Status,Field,Operator (Reported),Amended Date,Contact Name,Contact Phone,Operator Address,Operator City,Operator State,Operator Zip,Operator City 30mi,Section,Operator City 50mi,Township,Range,Block,Survey,True Vertical Depth UOM,Abstract,WGID,H2S Area,Surface Hole Latitude (WGS84),Surface Hole Longitude (WGS84),OFS Region,Bottom Hole Latitude (WGS84),Bottom Hole Longitude (WGS84),Lease Number,Permit Depth UOM,Permit Number,DI Basin,DI Play,DI Subplay,Operator Company Name,Operator Ticker
0,3000564345,SE,NaN,2019-12-13,2021-12-02,NM,CHAVES (NM),MAGNUM PROD,THE PEGGY FEDERAL,113,SAN ANDRES,2400.0,2400.0,NewDrill,OIL,V,PERMITTED,Active,RACE TRACK,"MAGNUM PRODUCING, LP",NaN,MATT GIROUX,405-289-9326,500 N. SHORELINE BLVD,CORPUS CHRISTI,TX,78401,NaN,13.0,NaN,09S,27E,NaN,NaN,ft,NaN,19309319,NaN,33.54,-104.14,PERMIAN BASIN,33.54,-104.14,326731,ft,TEMP86977488,NORTHWEST SHELF,NORTHWEST SHELF,NaN,"MAGNUM PRODUCING, LP",NaN
1,4245730898,3,2019-10-03,2019-10-08,2021-10-08,TX,TYLER (TX),MAGNUM PROD,BIG THICKET,1,"COCKFIELD,WILCOX,WILCOX 2-A,WILCOX 4-B,WILCOX ...",11800.0,11800.0,NewDrill,OIL & GAS,V,PERMITTED,Active,"HICKSBAUGH,THEUVENINS CREEK,WILDCAT","MAGNUM PRODUCING, LP",NaN,KEN AHUJA,3618823858,500 N SHORELINE BLVD STE 322,CORPUS CHRSTI,TX,78401,NaN,NaN,NaN,NaN,NaN,NaN,"BROWN, W S",ft,62.0,19299960,False,30.56,-94.34,GULF COAST,30.56,-94.34,NaN,ft,857731,GULF COAST CENTRAL,NaN,NaN,"MAGNUM PRODUCING, LP",NaN
2,4203933345,3,2019-07-03,2019-07-05,2021-07-05,TX,BRAZORIA (TX),MAGNUM PROD,DUCROZ,1,WILDCAT,11700.0,11700.0,NewDrill,OIL & GAS,V,SHUT-IN,Active,WILDCAT,"MAGNUM PRODUCING, LP",NaN,KEN AHUJA,3618823858,500 N SHORELINE BLVD STE 322,CORPUS CHRSTI,TX,78401,NaN,65.0,NaN,NaN,NaN,NaN,"DILLARD, N",ft,60.0,19286910,False,28.90,-95.60,GULF COAST,28.90,-95.60,27788,ft,854812,GULF COAST CENTRAL,NaN,NaN,"MAGNUM PRODUCING, LP",NaN
3,4232132426,3,2019-06-25,2019-06-26,2021-06-26,TX,MATAGORDA (TX),MAGNUM PROD,TRULL,1,WILDCAT,9000.0,9000.0,NewDrill,OIL & GAS,V,DRILLED,Active,WILDCAT,"MAGNUM PRODUCING, LP",NaN,KEN AHUJA,3618823858,500 N SHORELINE BLVD STE 322,CORPUS CHRSTI,TX,78401,NaN,NaN,NaN,NaN,NaN,NaN,"PERKINS, S",ft,371.0,19286199,False,28.77,-96.26,GULF COAST,28.77,-96.26,NaN,ft,854558,GULF COAST CENTRAL,NaN,NaN,"MAGNUM PRODUCING, LP",NaN
4,4246934444,2,2019-05-23,2019-05-24,2021-05-24,TX,VICTORIA (TX),MAGNUM PROD,SPACEK UNIT,1,WILDCAT,9800.0,9800.0,NewDrill,OIL & GAS,V,INACTIVE,Active,WILDCAT,"MAGNUM PRODUCING, LP",NaN,KEN AHUJA,3618823858,500 N SHORELINE BLVD STE 322,CORPUS CHRSTI,TX,78401,NaN,NaN,NaN,NaN,NaN,NaN,DAVID HINES,ft,175.0,19280561,False,29.08,-96.98,GULF COAST,29.08,-96.98,290069,ft,853438,GULF COAST WEST,NaN,NaN,"MAGNUM PRODUCING, LP",NaN


In [523]:
#finding which wells in production tables exist in the new drill permits table by magnum
prod["API/UWI"].isin(permits["API 10/UWI"]).head(10)

0     True
1     True
2     True
3     True
4    False
5     True
6     True
7    False
8    False
9    False
Name: API/UWI, dtype: bool

### Creating Column to Track Which Wells were Drilled By Magnum
* formula will check if api number exists in the newdrill permits data table

In [524]:
#creating new column representing whether or not well in production table exists
prod["Magnum Drilled (Y/N)"] = prod["API/UWI"].isin(permits["API 10/UWI"])

In [525]:
#mapping truths and falses to yes or no
prod["Magnum Drilled (Y/N)"] = prod["Magnum Drilled (Y/N)"].map({True: "Y", False: "N"})

In [526]:
#looking at how many wells are marked yes versus length of the entire permits table
prod["Magnum Drilled (Y/N)"].value_counts()


prod["Magnum Drilled (Y/N)"].value_counts()['Y']
len(permits)

print("There are {} permits not found in new drill permits table".format(len(permits)- prod["Magnum Drilled (Y/N)"].value_counts()['Y']))

N    542
Y    121
Name: Magnum Drilled (Y/N), dtype: int64

121

198

There are 77 permits not found in new drill permits table


## Counties Shapefile Cleanup

In [527]:
counties.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715..."
1,Ferry,Washington,53,019,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4..."
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0..."
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9..."
4,Pend Oreille,Washington,53,051,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9..."


In [528]:
len(counties)
counties["FIPS"].nunique()

3141

3141

### Reading in States Table Online with Postal Codes to Add to Counties Shapefile

In [529]:
stlink = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696'
stTable = pd.read_html(req.get(stlink).content)[0]

In [530]:
stTable.head()

,Name,Postal Code,FIPS
0,Alabama,AL,1.0
1,Alaska,AK,2.0
2,Arizona,AZ,4.0
3,Arkansas,AR,5.0
4,California,CA,6.0


### Joining Counties Shapefile with State Postal Codes and Cleaning up Duplicate Columns

In [531]:
counties = pd.merge(counties, stTable[["Name", "Postal Code"]], how = "left", left_on="STATE_NAME", right_on="Name")

In [532]:
counties.drop(columns = "Name", inplace=True)

In [533]:
counties.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN
1,Ferry,Washington,53,019,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4...",WA
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0...",WA
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9...",WA
4,Pend Oreille,Washington,53,051,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9...",WA


### Cleaning County Names by Converting to UpperCase for Matching/Joining as a Primary Key

In [534]:
counties["NAME"] = counties["NAME"].apply(lambda x: x.upper())

In [535]:
counties[["Postal Code", "NAME", "FIPS"]]

,Postal Code,NAME,FIPS
0,MN,LAKE OF THE WOODS,27077
1,WA,FERRY,53019
2,WA,STEVENS,53065
3,WA,OKANOGAN,53047
4,WA,PEND OREILLE,53051
5,ID,BOUNDARY,16021
6,MT,LINCOLN,30053
7,MT,FLATHEAD,30029
8,MT,GLACIER,30035
9,MT,TOOLE,30101


In [536]:
prod.head(1)

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N)
0,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,INACTIVE,V,9500.0,2019-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-25,NaN,3800.0,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9270.0,9312.0,0.66,46.3,2020-02-23,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",DAVID HINES,130836314,NaN,NaN,NaN,0,VICTORIA,NaN,Y


### Joining Production with Counties Data to Bring Over County FIPS Codes & Cleanup Duplicate Columns

In [537]:
prod = pd.merge(prod, counties[["Postal Code", "NAME", "FIPS"]], how = "left", left_on=["State", "Cleaned County"], right_on = ["Postal Code", "NAME"])

In [538]:
prod.drop(columns = ["Postal Code", "NAME"], inplace = True)

In [539]:
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
0,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,INACTIVE,V,9500.0,2019-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-25,NaN,3800.0,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9270.0,9312.0,0.66,46.3,2020-02-23,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",DAVID HINES,130836314,NaN,NaN,NaN,0,VICTORIA,NaN,Y,48469
1,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,INACTIVE,V,8560.0,2018-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5961.0,6044.0,NaN,30.5,2019-06-29,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27819,83.0,NaN,BIG HILL,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,373,USA,NaN,NaN,29.76,551,-94.26,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",EL&RR RR CO,130833332,NaN,NaN,NaN,0,JEFFERSON,NaN,Y,48245
2,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-11,1,NaN,NaN,NaN,27788,NaN,NaN,WILDCAT,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,65,USA,NaN,NaN,28.90,60,-95.60,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","DILLARD, N",130819038,NaN,NaN,NaN,0,BRAZORIA,NaN,Y,48039
3,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,2017-03-08,2020-01-01,2020-04-01,4.0,19409.0,160.0,687.0,6.0,144.0,6151.0,138.0,3922.0,NaN,197.0,3206.0,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,7.0,33.0,159.0,198448.0,29.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,23820.0,28251.82,28250.0,35.4,42.0,23.4,6151.0,4.0,197.0,1.0,1169.0,4.0,7.0,4.0,8781.0,8950.0,NaN,40.0,2020-01-09,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27736,169.0,NaN,"LUCKY, EAST",TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.96,45,-95.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","HALL, E",130785484,687.0,19409.0,138.0,0,MATAGORDA,1169.17,Y,48321
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP",

In [540]:
prod[prod["FIPS"].isna()]

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP","L & L TRANSPORTATION, LLC",NaN,NaN,NaN,"OAKLEY, NORTHWEST",LA,NaN,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, L.P.",NaN,130769684,119.0,NaN,0.0,801958,(N/A),NaN,N,NaN
217,0,"MAGNUM PRODUCING, LP",PAINTERS GAS UNIT,NaN,UNKNOWN (TX),3-G-1,GAS,INACTIVE,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,207020,NaN,0.0,MERCEDES,TX,04,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MAGNUM PRODUCING, LP",NaN,105561337,NaN,NaN,NaN,0,UNKNOWN,NaN,N,NaN


In [541]:
pd.options.display.max_columns = len(prod.columns)
prod.head()

,API/UWI,Operator Alias (Legacy),Well/Lease Name,Well Number,County/Parish,Reservoir,Production Type,Producing Status,Drill Type,Measured Depth (TD),Spud Date,First Prod Date,Last Prod Date,Months Produced,Cum Gas,Daily Gas,Cum Oil,Daily Oil,Latest Oil,Latest Gas,Cum Water,Cum BOE,DI Subplay,First Month Oil,First Month Gas,First 6 Oil,DI Basin,First 6 Gas,First 6 BOE,First 6 Water,DI Play,Prac IP Oil Daily,Prac IP BOE,Prac IP Gas Daily,Prac IP CFGED,Latest Water,Prior 12 Oil,Prior 12 Gas,Last Test Date,Prior 12 Water,Last Flow Pressure,Last WHSIP,2nd Month GOR,Latest GOR,Cum GOR,Last 12 Yield,2nd Month Yield,Latest Yield,Peak Gas,Peak Gas Month No.,Peak Oil,Peak Oil Month No.,Peak BOE,Peak BOE Month No.,Peak MMCFGE,Peak MMCFGE Month No.,Upper Perforation,Lower Perforation,Gas Gravity,Oil Gravity,Completion Date,Well Count,Max Active Wells,Gas Gatherer,Oil Gatherer,Lease Number,Gross Perforated Interval,True Vertical Depth,Field,State,District,AAPG Geologic Province,Section,Country,Township,Range,Surface Latitude (WGS84),Abstract,Surface Longitude (WGS84),Block,OCS Area,PGC Area,Operator (Reported),Survey,Entity ID,Last 12 Oil,Last 12 Gas,Last 12 Water,Other No.,Cleaned County,Latest BOE,Magnum Drilled (Y/N),FIPS
0,4246934444,"MAGNUM PRODUCING, LP",SPACEK UNIT,1,VICTORIA (TX),MIDDLE WILCOX,GAS,INACTIVE,V,9500.0,2019-06-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-25,NaN,3800.0,4600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9270.0,9312.0,0.66,46.3,2020-02-23,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",290069,42.0,NaN,HELEN GOHLKE,TX,02,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,29.08,175,-96.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",DAVID HINES,130836314,NaN,NaN,NaN,0,VICTORIA,NaN,Y,48469
1,4224532864,"MAGNUM PRODUCING, LP",MCLEAN UNIT 4,4,JEFFERSON (TX),HETEROSTEGINA - DISPOSAL,OIL,INACTIVE,V,8560.0,2018-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5961.0,6044.0,NaN,30.5,2019-06-29,1,NaN,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27819,83.0,NaN,BIG HILL,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,373,USA,NaN,NaN,29.76,551,-94.26,NaN,NaN,P-330,"MAGNUM PRODUCING, LP",EL&RR RR CO,130833332,NaN,NaN,NaN,0,JEFFERSON,NaN,Y,48245
2,4203933345,"MAGNUM PRODUCING, LP",DUCROZ,1,BRAZORIA (TX),TOP OF FRIO,OIL,INACTIVE,V,11100.0,2019-10-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-11,1,NaN,NaN,NaN,27788,NaN,NaN,WILDCAT,TX,03,TEXAS & LOUISIANA GULF COAST BASIN,65,USA,NaN,NaN,28.90,60,-95.60,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","DILLARD, N",130819038,NaN,NaN,NaN,0,BRAZORIA,NaN,Y,48039
3,4232132401,"MAGNUM PRODUCING, LP",ANDERSON,3,MATAGORDA (TX),HUEBNER,OIL,INACTIVE,D,10600.0,2017-03-08,2020-01-01,2020-04-01,4.0,19409.0,160.0,687.0,6.0,144.0,6151.0,138.0,3922.0,NaN,197.0,3206.0,NaN,GULF COAST CENTRAL,NaN,NaN,NaN,NaN,7.0,33.0,159.0,198448.0,29.0,NaN,NaN,2020-01-22,NaN,NaN,NaN,23820.0,28251.82,28250.0,35.4,42.0,23.4,6151.0,4.0,197.0,1.0,1169.0,4.0,7.0,4.0,8781.0,8950.0,NaN,40.0,2020-01-09,1,1.0,"MAGNUM PRODUCING, LP","GULFMARK ENERGY, INC.",27736,169.0,NaN,"LUCKY, EAST",TX,03,TEXAS & LOUISIANA GULF COAST BASIN,NaN,USA,NaN,NaN,28.96,45,-95.98,NaN,NaN,P-330,"MAGNUM PRODUCING, LP","HALL, E",130785484,687.0,19409.0,138.0,0,MATAGORDA,1169.17,Y,48321
4,0,"MAGNUM PRODUCING, LP",LA LUW 801958,NaN,(N/A) (LA),NaN,OIL,INACTIVE,U,NaN,NaN,2010-11-01,2012-01-01,2.0,NaN,NaN,322.0,NaN,119.0,NaN,0.0,322.0,NaN,203.0,NaN,203.0,NaN,0.0,203.0,0.0,NaN,4.0,4.0,0.0,23032.0,0.0,203.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,203.0,1.0,203.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,1.0,"GULF SOUTH PIPELINE COMPANY, LP",

## Exporting Prepared Production Table to File

In [542]:
prod.to_excel("Prepared Magnum Production Table.xlsx", index = False)